<a href="https://colab.research.google.com/github/HyeJin816/ESAA_22/blob/main/0515_ch9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ch9. 추천시스템**

## **01 추천 시스템 개요**

사용자가 스스로도 몰랐던 취향을 시스템이 발견하고 그에 맞는 콘텐츠를 추천. 해당 시스템을 신뢰하게 될수록 더 많은 데이터가 추천 시스템에 축적되면서 추천이 더욱 정확해지고 좋은 선순환 시스템 구축. 

### **온라인 스토어의 필수 요소, 추천시스템**

온라인 스토어는 많은 양과 고객과 상품 관련 데이터를 가지고 있으며 이 모든 데이터가 사용자의 흥미를 돋을 상품을 즉각적으로 추천하는데 사용.
> 사용자가 어떤 상품을 구매했는가?  
> 사용자가 어떤 제품을 browse했는가?  
> 사용자가 무엇을 클릭했는가?  
> 사용자의 제품 평점은?

### **추천시스템의 유형**

* 콘텐츠 기반 필터링
* 협업필터링
  * 최근접 이웃 협업 필터링
  * 잠재 요인 협업 필터링

## **02 콘텐츠 기반 필터링 추천 시스템**

사용자가 특정한 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식.

## **03 최근접 이웃 협업 필터링**

사용자 행동 양식(아이템에 매긴 평점 정보, 상품 구매 이력)만을 기반으로 추천을 수행하는 것  
: 사용자-아이템 평점 매트릭스와 같은 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측 평가 하는 것.  
> **사용자 기반 User-User** : 특성 사용자와 유사한 다른 사용자를 선정하여 그 사용자가 좋아하는 아이템 추천 방식  
> **아이템 기반 Item-Item** : 아이템이 갖는 속성과 상관없이 사용자들이 그 아이템에 대한 호감의 평가 척도가 유사한 아이템을 추천하는 기준이 되는 알고리즘

## **04 잠재 요인 협업 필터링**

'잠재 요인'이 어떤 것인지 명확히 정의할 수는 없지만 user-item 평점 행렬 데이터로 잠재 요인을 끄집어 내는 것을 의미

### **행렬 분해의 이해**

다차원 매트릭스를 저차원 매트릭스로 분해하는 기법  
* **SVD** Singular Vector Decompositon : NaN값이 없는 행렬의 분해  
* **NMF** Non-Negative Matrix Factorization

### **확률적 경사 하강법을 이용한 행렬 분해**

예측 행렬의 값이 실제 행렬 값과 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 행렬 유추

In [1]:
import numpy as np

#원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 K는 3으로 설정
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN,],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 1, np.NaN]])

num_users, num_items = R.shape
K = 3

# P와 Q 행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [2]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    #두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    #실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

SGD 기반 행렬 분해 수행

In [3]:
# R>0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
      # 실제 값과 예측 값의 차이인 오류 값 구함
      eij = r- np.dot(P[i, :], Q[j, :].T)
      # Regulariation을 반영한 SGD 업데이트 공식 적용
      P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
      Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j, :])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0:
        print("### iteration step: ", step," rmse: ", rmse)

### iteration step:  0  rmse:  3.2011202085938044
### iteration step:  50  rmse:  0.6154267829430629
### iteration step:  100  rmse:  0.2536345481699502
### iteration step:  150  rmse:  0.12492903462065194
### iteration step:  200  rmse:  0.06275532232130014
### iteration step:  250  rmse:  0.03506384672845852
### iteration step:  300  rmse:  0.02389785844276385
### iteration step:  350  rmse:  0.019716500751376057
### iteration step:  400  rmse:  0.018154210025261398
### iteration step:  450  rmse:  0.017522410099519235
### iteration step:  500  rmse:  0.017227673409515896
### iteration step:  550  rmse:  0.01706152015024207
### iteration step:  600  rmse:  0.016947590721601208
### iteration step:  650  rmse:  0.016856507138095295
### iteration step:  700  rmse:  0.016776412354044923
### iteration step:  750  rmse:  0.016702292684141938
### iteration step:  800  rmse:  0.016631926279262455
### iteration step:  850  rmse:  0.01656427805855475
### iteration step:  900  rmse:  0.01649883

In [5]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬: \n', np.round(pred_matrix, 3))

예측 행렬: 
 [[3.991 1.    1.525 1.999 1.664]
 [7.926 4.978 1.8   2.98  1.003]
 [5.46  0.163 2.989 3.98  3.985]
 [4.969 2.007 0.999 1.023 0.365]]
